In [46]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException
from matplotlib import style
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
import os

In [23]:
print(plt.style.available)
%matplotlib inline
chrome_options = Options()
chrome_options.page_load_strategy = 'eager'  # 设置加载策略为 eager
driver = webdriver.Chrome(options=chrome_options)

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [24]:
url_esf='https://esf.fang.com/'
driver.get(url_esf)

In [25]:
Haidian_esf=driver.find_element(By.XPATH,'/html/body/div[4]/div[3]/div[2]/div[1]/ul/li[1]/ul/li[2]/a')
Haidian_esf.click()

In [26]:
Shangdi_esf=driver.find_element(By.XPATH,'/html/body/div[4]/div[3]/div[2]/div[1]/ul/li[2]/ul/li[17]/a')
driver.execute_script("arguments[0].click();", Shangdi_esf)

In [27]:
#mine data on esf website in 20 pages
i = 0
Num_Pages = 20
house_esf_data=[]
while i < Num_Pages:
    location='上地'
    house_items=driver.find_elements(By.CSS_SELECTOR, "div.shop_list dl.clearfix")
    for item in house_items:
        try:
            area_element = item.find_element('class name','tel_shop')
            area_text = area_element.text
            import re
            area_match = re.search(r'(\d+\.?\d*)\s*㎡', area_text)
            area = area_match.group(1) if area_match else None
            
            community_element = item.find_element(By.CSS_SELECTOR, "p.add_shop a[title]")
            community_name = community_element.get_attribute('title') if community_element else None
        

            price_element = item.find_element(By.CSS_SELECTOR, "dd.price_right span:not(.red)")
            unit_price = price_element.text if price_element else None
            price_match1 = re.search(r'(\d+\.?\d*)\s*元/㎡', unit_price)
            unit_price = price_match1.group(1) if price_match1 else None

            total_price_element = item.find_element(By.CSS_SELECTOR, "dd.price_right span.red b")
            total_price = total_price_element.text if total_price_element else None
            price_match2 = re.search(r'(\d+\.?\d*)\s*', total_price)
            total_price = float(price_match2.group(1)) if price_match2 else None
            total_price=float(total_price)*10000
            
            house_esf_data.append({
                    'location': location,
                    'community':community_name,
                    'square': area,
                    'total':total_price,
                    'unit': unit_price

                })
        except Exception as e:
                print(f"fail extracting information: {e}")
                continue
    try:
        next_page = driver.find_element(By.LINK_TEXT, '下一页')
        driver.execute_script("arguments[0].click();", next_page)
        i += 1
    except NoSuchElementException:
        break

df_esf = pd.DataFrame(house_esf_data)

In [28]:
df_esf.head()

,location,community,square,total,unit
0,上地,上地西里,125.3,10990000.0,87709
1,上地,上第MOMA,191,12600000.0,65968
2,上地,上地佳园,149.73,19500000.0,130234
3,上地,上第MOMA,108.82,6980000.0,64142
4,上地,上第MOMA,146.25,10680000.0,73025


In [29]:
df_esf.describe()

,total
count,1.200000e+03
mean,9.835546e+06
std,3.752118e+06
min,2.450000e+06
25%,7.500000e+06
50%,9.025000e+06
75%,1.188000e+07
max,2.980000e+07


In [30]:
df_esf[['square','unit']].describe()

,square,unit
count,1200,1200
unique,283,459
top,111.57,69176
freq,24,14


In [31]:
#open renting house website
url_rent='https://zu.fang.com'
driver.get(url_rent)

In [32]:
Haidian_rent=driver.find_element(By.XPATH,'/html/body/div[4]/div[2]/div[2]/div/dl[1]/dd/a[3]')
driver.execute_script("arguments[0].click();", Haidian_rent)

In [33]:
Shangdi_rent=driver.find_element(By.XPATH,'//*[@id="rentid_D04_08"]/a[20]')
driver.execute_script("arguments[0].click();", Shangdi_rent)

In [34]:
# mine data on rent website in 20 pages
j = 0
Num_Pages = 20
house_rent_data=[]
while j < Num_Pages:
    location='上地'
    house_items=driver.find_elements(By.CSS_SELECTOR, "div.houseList dl.list.hiddenMap.rel")
    for item in house_items:
        try:
            area_element = item.find_element(By.CSS_SELECTOR,'p.font15.mt12.bold')
            area_text = area_element.text
            import re
            area_match = re.search(r'(\d+\.?\d*)\s*㎡', area_text)
            area = area_match.group(1) if area_match else None

            community_element = item.find_element(By.CSS_SELECTOR, "span")
            community_name = community_element.text if community_element else None
            
            price_element = item.find_element('class name', "price")
            price_permonth = price_element.text if price_element else None

            house_rent_data.append({
                    'location': location,
                    'community':community_name,
                    'square': area,
                    'rent': price_permonth
                })
        except Exception as e:
                print(f"fail extracting information: {e}")
                continue
    try:
        next_page = driver.find_element(By.LINK_TEXT, '下一页')
        driver.execute_script("arguments[0].click();", next_page)
        j += 1
    except NoSuchElementException:
        break

df_rent = pd.DataFrame(house_rent_data)

In [35]:
df_rent.head()

,location,community,square,rent
0,上地,|,314,25500
1,上地,|,114,10500
2,上地,|,125,11000
3,上地,|,106,12000
4,上地,|,136,12900


In [36]:
df_rent.describe()

,location,community,square,rent
count,950,950,950,950
unique,1,2,131,166
top,上地,|,59,12000
freq,950,734,32,50


In [47]:
if not os.path.exists('data'):
    os.makedirs('data')

In [48]:
df_esf.to_excel('data/df_esf_Shangdi.xlsx', index=False)
df_rent.to_excel('data/df_rent_Shangdi.xlsx', index=False)

In [49]:
df_esf.to_csv('data/df_esf_Shangdi.csv', index=False)
df_rent.to_csv('data/df_rent_Shangdi.csv', index=False)